#              In C - Working notes



## A - Basic plan

### Phase 1 - Minimal working prototype

| Step                                                                  | Progress |
|-----------------------------------------------------------------------|----------|

1. Write general program structure as functions                          DONE
2. Write patterns-generating function                                    DONE (with all 53 patterns)
3. Write performance generating function:
      - General setup                                                    DONE
      - Create individual *randomized* scores for player                 DONE //but as a beforehand computation
        according to performance parameters
      - Add random cell transposition
      - Create instruments and assign to players:
        - As internal SynthDefs, as MIDI, as samples, etc.

4. Write performance executing function(s):
      - As simple as possible with internal default PBind instrument     DONE
      - Slightly more complex with different internal instruments
      - With MIDI out to synth (possibly Carla-hosted Kontakt)
        Check out Linux Utility Quark to easy connection job to Carla
      - With MIDI out to Ardour (controlled from Ardour)


### Phase 2 - Refactor into classes


### Phase 3 - Add complete scores and full instrument set
    Complete score added already in phase 1

### Phase 4 - Add Riley's additional (and not so additional) options:

- Pulse instrument
- Improvised percussions
- Ending with repeat crescendo and diminuendo and instruments' randomly dropping off


### Phase 5 - Add some spatialization

### Phase 6 - Explore live performance interaction with algorithm

### Phase 7 - Add minimal GUI to control performance at run time

## B -  Scattered notes

### 1. How to play a sampled instrument?

- One (rather low-level) possibility: use **BufFiles** quark extension: 
   > "An easy way to load a bunch of audio files into easily accessible collections of buffers in Supercollider. Great for sample libraries. This class loads the sound files at the root of a folder into the array, ignoring any subfolders." 
   See help file for details.

- Another possibility is to use the **SuperSampler** quark, which is way more sophisticated (and also, consequently, harder to use and easier to break things with...)

#### 1.1 More on (non-sampled) instruments:

- Check out the SynthDefs listed on the resource page curated by Mads Kjeldgaard: https://github.com/cleinias/awesome-supercollider#synthdefs
  Among the many Synths listed there there are some simple ones like piano, harpsichord, strings, etc.

- More instrument possibilities: use the physical models from the **STK (Sound ToolKit)** availabe in SC as STK+NameOfInstrument. According to the recent book/tutorial by Thor Magnusson ([Scoring Sound](https://thormagnusson.gitbooks.io/scoring/content/index.html)), the port of STK's instrument to SC is old and buggy, though. Besides, it relies on an older version of stk which misses at least a couple of instruments (recorder and guitar, among them). Here is Magnusson's own example on how to use one of the generic stk instruments to produce a xylophone SynthDef:

In [16]:
((s.boot;);
SynthDef(\xylo, { |out=0, freq=440, gate=1, amp=0.3, sustain=0.5, pan=0|
    var sig = StkBandedWG.ar(freq, instr:1, mul:3);
    var env = EnvGen.kr(Env.adsr(0.0001, sustain, sustain, 0.3), gate, doneAction:2);
    Out.ar(out, Pan2.ar(sig, pan, env * amp));
}).add;

Synth(\xylo);
a = Pbind(\instrument, \xylo, \freq, Pseq(({|x|x+60}!13).mirror).midicps, \dur, 0.2).play;)

#### 1.2 Instruments as MIDI output

TBW yet

### Instruments and PBind
(See SC's _Patterns guide_, sec. 3 for extended discussion, plus SC's book. Here is the basic gist)

Pbind's 'play' method creates an event and plays according to an event prototype:

- using a default SynthDef, and
- passing to the SynthDef all the *indentically named* _keys_ passed to the PBind itself (but see caveats in docs)
- to change instruments, you define an appropriate SynthDef and then pass its symbol to the PBind with the `\instrument` key.

Example from the docs:

In [19]:
SynthDef(\harpsi, { |outbus = 0, freq = 440, amp = 0.1, gate = 1|
    var out;
    out = EnvGen.ar(Env.adsr, gate, doneAction: Done.freeSelf) * amp *
        Pulse.ar(freq, 0.25, 0.75);
    Out.ar(outbus, out ! 2);
}).add;    // see below for more on .add

In [20]:
p = Pbind(
        // Use \harpsi, not \default
    \instrument, \harpsi,
    \degree, Pseries(0, 1, 8),
    \dur, 0.25
).play;

## 2. Change players to be instruments (or instantiated on basis of instruments)

Instead of passing an integer to the perofrmance function, we should pass an array of instruments (either randomly chosen, or based on premade collections, which may inlude a sequenec of identical instruments). The number of player is thus, trivially, the size of the array. 
May want to add a check on the array to make sure it contains all SynthDefs (and default to all pianos or some such otherwise).  